# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning I</font>

### Mini-Projeto 5 - Reconhecimento e Classificação de Idioma em Arquivos de Áudio Usando CNNs

Este notebook treina uma rede neural convolucional para classificar arquivos de áudio de gravações de voz nos idiomas falados. O conjunto de dados utilizado contém 65.000 arquivos em 176 idiomas e está disponível para download na página de uma competição em Data Science no site TopCoder (https://goo.gl/G5XBJl). É interessante ver que as CNNs funcionam bem em problemas onde a intuição não leva você a lugar nenhum.

Obs: O dataset com os arquivos de áudio possui 4.6 GB.

In [1]:
import tensorflow as tf
tf.__version__

'2.1.0'

In [2]:
import keras as k
k.__version__

Using TensorFlow backend.


'2.3.1'

In [ ]:
#!pip install librosa

In [ ]:
# Imports
import os
import h5py
import glob
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
import librosa as lr
import dask.array as da

from keras.models import Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.layers import Dropout, Input, BatchNormalization
from keras.optimizers import Nadam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils

In [ ]:
in_dim = (192,192,1)
out_dim = 176
batch_size = 32
mp3_path = 'data/mp3/'
tr_path = 'data/train/'
va_path = 'data/valid/'
te_path = 'data/test/'
data_size = 66176
tr_size = 52800
va_size = 4576
te_size = 8800

Isso converterá um único arquivo mp3 em um espectrograma e retornará a imagem. O mel-spectrogram é usado para obter mais informações nas frequências mais baixas, semelhantes à audição humana. As intensidades e as frequências são então escalonadas logaritmicamente. Esta função também cortará 5% do começo e fim do arquivo. Isso é para se livrar do silêncio e garantir as mesmas dimensões de cada arquivo gerado. A conversão leva cerca de 1 segundo por minuto de áudio.

In [ ]:
def mp3_to_img(path, height=192, width=192):
    signal, sr = lr.load(path, res_type='kaiser_fast')
    
    # Isso vai cortar 5% do começo e do fim
    hl = signal.shape[0]//(width*1.1)
    spec = lr.feature.melspectrogram(signal, n_mels=height, hop_length=int(hl))
    img = lr.logamplitude(spec)**2
    start = (img.shape[1] - width) // 2
    return img[:, start:start+width]

Converter todos os arquivos mp3 para espectrograma jpgs. A função process_audio_with_classes() usará os rótulos para classificar todos os jpgs em subpastas correspondentes. Isso é útil para a função flow_from_directory no Keras.

In [ ]:
def process_audio(in_folder, out_folder):
    os.makedirs(out_folder, exist_ok=True)
    files = glob.glob(in_folder+'*.mp3')
    start = len(in_folder)
    for file in files:
        img = mp3_to_img(file)
        sp.misc.imsave(out_folder + file[start:] + '.jpg', img)
        
def process_audio_with_classes(in_folder, out_folder, labels):
    os.makedirs(out_folder, exist_ok=True)
    for i in range(len(labels['Sample Filename'])):
        file = labels['Sample Filename'][i]
        lang = labels['Language'][i]
        os.makedirs(out_folder + lang, exist_ok=True)
        img = mp3_to_img(in_folder+file)
        sp.misc.imsave(out_folder + lang + '/' + file + '.jpg', img)

Converte um diretório de imagens em um arquivo HDF5 armazenando as imagens em uma matriz. A forma da matriz será (img_num, height, width [, channels]).

In [ ]:
def jpgs_to_h5(source, target, name):
    da.image.imread(source + '*.jpg').to_hdf5(target, name)

## Preparando os Dados

Os dados brutos consistem em 66176 44kHz estéreo mp3 com um comprimento de 10 segundos cada. O conjunto de dados é perfeitamente balanceado com 376 arquivos por idioma.

Podemos visualizar esse arquivo convertendo-o em um log-mel-spectrogram.

- O eixo y mostra a frequência
- O eixo x mostra a hora
- O cor mostra a intensidade de uma frequência em um determinado momento

Embora seja fácil ler um espectrograma, é difícil julgar "intuitivamente" o conteúdo de um arquivo de áudio. Nossa primeira tentativa de convertê-los pareceu totalmente boa, mas não conseguimos treinar a rede. Usamos um espectrograma regular que depois convertemos para frequências em escala de log. O problema era que, espremendo as frequências mais altas, estávamos separando as frequências mais baixas. Essa é a ideia geral de uma escala de log, mas não levamos em conta que a resolução nas frequências mais baixas sofreria muito. 

Começamos com uma resolução de 224x448 pixels, mas isso levou uma eternidade. Aplicamos alguns redimensionamentos assimétricos e notamos que a suposição de reservar mais espaço para o eixo do tempo estava errada. Imagens quadradas pareciam ter melhor desempenho. Então, seguimos em frente e convertemos tudo para 192x192, o que não prejudicou muito o desempenho.

A "verificação de sanidade" dos dados acabou por ser difícil com este conjunto de dados. Aparentemente, você pode ter 176 idiomas diferentes sem incluir inglês, alemão ou francês. Mas todas as amostras holandesas soavam como o som das pessoas holandesas, então imaginamos que não poderia ser tão errado assim.

In [ ]:
# Converte os arquivos mp3 para jpgs.
process_audio('data/mp3/', 'data/jpg/')

In [ ]:
# Converte a pasta de imagens em um arquivo contêiner compactado.
jpgs_to_h5('data/jpg/', 'data/data.h5', 'data')

In [ ]:
# Embaralhe os dados e divida-os em treino, validação e teste
y = pd.read_csv('data/train_list.csv')['Language']
y = pd.get_dummies(y)
y = y.reindex_axis(sorted(y.columns), axis=1)
y = y.values
y = da.from_array(y, chunks=1000)
y

In [ ]:
x = h5py.File('data/data.h5')['data']
x = da.from_array(x, chunks=1000)
x

In [ ]:
shfl = np.random.permutation(data_size)

tr_idx = shfl[:tr_size]
va_idx = shfl[tr_size:tr_size+va_size]
te_idx = shfl[tr_size+va_size:]

x[tr_idx].to_hdf5('data/x_tr.h5', 'x_tr')
y[tr_idx].to_hdf5('data/y_tr.h5', 'y_tr')
x[va_idx].to_hdf5('data/x_va.h5', 'x_va')
y[va_idx].to_hdf5('data/y_va.h5', 'y_va')
x[te_idx].to_hdf5('data/x_te.h5', 'x_te')
y[te_idx].to_hdf5('data/y_te.h5', 'y_te')

## Carregando e Processando os Dados

In [ ]:
# Ler os dados que preparamos e verificar suas dimensões

x_tr = da.from_array(h5py.File('data/x_tr.h5')['x_tr'], chunks=1000)
y_tr = da.from_array(h5py.File('data/y_tr.h5')['y_tr'], chunks=1000)
print(x_tr.shape, y_tr.shape)

x_va = da.from_array(h5py.File('data/x_va.h5')['x_va'], chunks=1000)
y_va = da.from_array(h5py.File('data/y_va.h5')['y_va'], chunks=1000)
print(x_va.shape, y_va.shape)

x_te = da.from_array(h5py.File('data/x_te.h5')['x_te'], chunks=1000)
y_te = da.from_array(h5py.File('data/y_te.h5')['y_te'], chunks=1000)
print(x_te.shape, y_te.shape)

In [ ]:
x_tr /= 255.
x_va /= 255.
x_te /= 255.

In [ ]:
# Vamos verificar uma amostra só para ter certeza
test_img = x_tr[0, :, :, 0]
plt.imshow(test_img)
plt.show()

## Modelo

Tentamos cerca de 30 modelos diferentes com foco em arquiteturas mais recentes, como redes residuais, redes em redes, espremendo e expandindo convoluções, mas no final um 5x-Conv-MaxPool funcionou melhor. Nós queríamos substituir as últimas camadas Dense por AveragePooling. Elas dão um pouco mais de visão do que está acontecendo em comparação com o modelo "caixa preta" que resulta de camadas densas. No entanto, não funcionou bem. Estamos supondo que isso é porque os espectrogramas mostram uma abstração diferente da informação em comparação com uma foto regular mostrando um objeto.

Nos testes o uso de Elu substituiu a necessidade de Normalização em Lote, que normalmente melhora qualquer modelo. Não os incluímos por motivos de desempenho.

In [ ]:
i = Input(shape=in_dim)
m = Conv2D(16, (3, 3), activation='elu', padding='same')(i)
m = MaxPooling2D()(m)
m = Conv2D(32, (3, 3), activation='elu', padding='same')(m)
m = MaxPooling2D()(m)
m = Conv2D(64, (3, 3), activation='elu', padding='same')(m)
m = MaxPooling2D()(m)
m = Conv2D(128, (3, 3), activation='elu', padding='same')(m)
m = MaxPooling2D()(m)
m = Conv2D(256, (3, 3), activation='elu', padding='same')(m)
m = MaxPooling2D()(m)
m = Flatten()(m)
m = Dense(512, activation='elu')(m)
m = Dropout(0.5)(m)
o = Dense(out_dim, activation='softmax')(m)

model = Model(inputs=i, outputs=o)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Nadam(lr=1e-3), metrics=['accuracy'])
model.fit(x_tr, y_tr, epochs=2, verbose=1, validation_data=(x_va, y_va))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Nadam(lr=1e-4), metrics=['accuracy'])
model.fit(x_tr, y_tr, epochs=3, verbose=1, validation_data=(x_va, y_va))

In [ ]:
# Carregando o modelo
model = load_model('speech_v9.h5')

In [ ]:
# Avaliando o desempenho
model.evaluate(x_te, y_te)

## Fim